## Word Vectorisation Notebook
Below is the code used for generating various embeddings used in the neural models.

### Imported libraries

In [4]:
import os
import urllib.request
import numpy as np
import zipfile
import tensorflow as tf
import collections
import random
import h5py as h5py


from keras.models import Sequential,Model
from keras.optimizers import RMSprop
from keras.layers import Embedding,LSTM,Dense,Lambda,merge,Input
from keras.callbacks import TensorBoard,ModelCheckpoint,Callback
from keras import backend as K

/Applications/anaconda/envs/nlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


### Functions defined
**maybe_download(filename)**
Download a file if not present.

**read_data(filename)**
Extract the first file enclosed in a zip file as a list of words.

**build_dataset(words, n_words)**
Process raw inputs into a dataset.

**generate_batches(data, size, contextWidth, negativeSize)**
Returns batches of input words with their contexts and a set of negative samples.

In [5]:
def maybe_download(url, filename):
  """Download a file if not present."""
  if not os.path.exists("./downloads/"+filename):
    filename, _ = urllib.request.urlretrieve(url + filename, "./downloads/"+filename)
  return filename

def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile("./downloads/"+filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

def generate_batches(data, size, contextWidth, negativeSize):
  cHalfWidth = int(contextWidth/2)
  words = []
  contexts = []
  negatives = []
  index = random.sample(range(cHalfWidth,len(data)-cHalfWidth),size)
  for z in index:
      context = []
      for m in range(-cHalfWidth,cHalfWidth+1):
        if m == 0:
          words.append([data[z]])
        else: 
          context.append(data[z+m])
      contexts.append(context)
      negatives.append(random.sample(data,negativeSize))
  return([np.array(words),np.array(contexts),np.array(negatives)],[np.array([1]*size),np.array([[0]*negativeSize]*size)])

### Downloading Wikipedia text database

In [6]:
filename = maybe_download('http://mattmahoney.net/dc/', 'text8.zip')
vocabulary = read_data(filename)
print('Number of words: ', len(vocabulary))

Number of words:  17005207


### Parameters for skipgram model

In [7]:
vocabulary_size = 5000
data_index = 0
batch_size = 128
wordvec_dim = 32

skip_window = 3       # How many words to consider left and right.
num_skips = 4         # How many times to reuse an input to generate a label.
context_half = 3
context_size = context_half*2

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
neg_size = 5    # Number of negative examples to sample.


### Creating a dictionary and reverse dictionary for word embedding

In [8]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.

### Saving a .tsv label file for viewing in TensorBoard

In [9]:
with open('./logs/word2vec_label.tsv', 'w') as fr:
    for i in range(vocabulary_size):
        fr.write(reverse_dictionary[i]+'\n')

### Generate training/validation batches

In [10]:
X,Y = generate_batches(data, 500000, context_size, neg_size)
vX, vY = generate_batches(data, 5000, context_size, neg_size)

### Definining neural model with Keras

include image of cbow graph.


In [14]:
word = Input(shape=(1,), name='inputWord')
context = Input(shape=(context_size,), name='inputContext')
negSamples = Input(shape=(neg_size,), name='inputNegatives')

word2vec = Embedding(input_dim=vocabulary_size,output_dim=wordvec_dim, embeddings_initializer='glorot_normal', name='word2vec')

vec_word = word2vec(word)
vec_context = word2vec(context)
vec_negSamples = word2vec(negSamples)
cbow = Lambda(lambda x: K.mean(x, axis=1), name='cbowAverage')(vec_context)

word_context = merge([vec_word, cbow], mode='dot')
negative_context = merge([vec_negSamples, cbow], mode='dot', concat_axis=-1)

model = Model(input=[word,context,negSamples], output=[word_context,negative_context])

/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  from i

In [12]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputWord (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
inputContext (InputLayer)       (None, 6)            0                                            
__________________________________________________________________________________________________
word2vec (Embedding)            multiple             160000      inputWord[0][0]                  
                                                                 inputContext[0][0]               
                                                                 inputNegatives[0][0]             
__________________________________________________________________________________________________
inputNegat

### Create logs for saving parameters and run training

In [13]:
tensorboard = TensorBoard(log_dir='./logs/wordvec', 
  batch_size=500, histogram_freq=1, write_images=True, write_grads=False, write_graph=True, embeddings_freq=1)

model_checkpoint = ModelCheckpoint('./logs/model.hdf5')

model.fit(X,Y,epochs=50,batch_size=500,callbacks=[model_checkpoint,tensorboard], validation_data=(vX,vY))

Train on 500000 samples, validate on 5000 samples
Epoch 1/50
500000/500000 [==============================] - 7s 13us/step - loss: 0.5458 - merge_1_loss: 0.3781 - merge_2_loss: 0.1677 - merge_1_acc: 0.4066 - merge_2_acc: 0.2316 - val_loss: 0.4843 - val_merge_1_loss: 0.2907 - val_merge_2_loss: 0.1936 - val_merge_1_acc: 0.4934 - val_merge_2_acc: 0.2272
Epoch 2/50
500000/500000 [==============================] - 7s 15us/step - loss: 0.4699 - merge_1_loss: 0.2734 - merge_2_loss: 0.1965 - merge_1_acc: 0.5208 - merge_2_acc: 0.2268 - val_loss: 0.4649 - val_merge_1_loss: 0.2673 - val_merge_2_loss: 0.1976 - val_merge_1_acc: 0.5320 - val_merge_2_acc: 0.2236
Epoch 3/50
500000/500000 [==============================] - 8s 16us/step - loss: 0.4544 - merge_1_loss: 0.2613 - merge_2_loss: 0.1931 - merge_1_acc: 0.5390 - merge_2_acc: 0.2266 - val_loss: 0.4532 - val_merge_1_loss: 0.2583 - val_merge_2_loss: 0.1949 - val_merge_1_acc: 0.5486 - val_merge_2_acc: 0.2240
Epoch 4/50
500000/500000 [===============

Epoch 28/50
500000/500000 [==============================] - 13s 27us/step - loss: 0.3678 - merge_1_loss: 0.1975 - merge_2_loss: 0.1702 - merge_1_acc: 0.6903 - merge_2_acc: 0.2295 - val_loss: 0.4100 - val_merge_1_loss: 0.2326 - val_merge_2_loss: 0.1774 - val_merge_1_acc: 0.6262 - val_merge_2_acc: 0.2254
Epoch 29/50
500000/500000 [==============================] - 11s 21us/step - loss: 0.3672 - merge_1_loss: 0.1971 - merge_2_loss: 0.1701 - merge_1_acc: 0.6915 - merge_2_acc: 0.2292 - val_loss: 0.4101 - val_merge_1_loss: 0.2333 - val_merge_2_loss: 0.1768 - val_merge_1_acc: 0.6260 - val_merge_2_acc: 0.2270
Epoch 30/50
500000/500000 [==============================] - 9s 18us/step - loss: 0.3667 - merge_1_loss: 0.1965 - merge_2_loss: 0.1701 - merge_1_acc: 0.6928 - merge_2_acc: 0.2293 - val_loss: 0.4106 - val_merge_1_loss: 0.2317 - val_merge_2_loss: 0.1789 - val_merge_1_acc: 0.6316 - val_merge_2_acc: 0.2266
Epoch 31/50
500000/500000 [==============================] - 12s 24us/step - loss: 0.3